## Homework - week 2

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and model management.

In [19]:
! mlflow --version

mlflow, version 2.12.2


In [26]:
# Q2. Download and preprocess the data

import os
import requests

data_dir = '../data/hw-week-2'
datasets = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet'
]

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

for dataset in datasets:
    filename = dataset.split('/')[-1]
    dst = f'{data_dir}/{filename}'
    
    if os.path.isfile(dst):
        print(f"File {filename} already exists")
    else:
        print(f"Downloading {filename}...")
        response = requests.get(dataset)
    
        with open(dst, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"Downloading {filename} complete")


File green_tripdata_2023-02.parquet already exists
File green_tripdata_2023-03.parquet already exists


In [44]:
# Use the script preprocess_data.py located in the folder homework to preprocess the data.

# The script will:

#     load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
#     fit a DictVectorizer on the training set (January 2023 data),
#     save the preprocessed datasets and the DictVectorizer to disk.

! mkdir -p ../output/ && rm -f ../output/* 2>/dev/null 
! python ./scripts/hw-week-2/preprocess_data.py --raw_data_path ../data/hw-week-2 --dest_path ../output
! echo "Num of files: $(ls -1q ../output/ | wc -l)"

Num of files: 4


In [51]:
# Q3. Train a model with autolog
# We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.
! python ./scripts/hw-week-2/train.py --data_path ../output

# min_samples_split = 2

2024/05/27 14:53:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/05/27 14:53:15 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2024/05/27 14:53:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2024/05/27 14:53:16 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
# Q4. Launch the tracking server locally
# In addition to backend-store-uri, what else do you need to pass to properly configure the server?
# ** `--default-artifact-route`

In [17]:
# Q5. Tune model hyperparameters

# Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.
# What's the best validation RMSE that you got?
! python ./scripts/hw-week-2/hpo.py --data_path ../output

# Best rmse: 5.335

  0%|                                    | 0/15 [00:00<?, ?trial/s, best loss=?]/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

  7%|▋          | 1/15 [00:06<01:33,  6.66s/trial, best loss: 5.370086069268862]/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

 13%|█▍         | 2/15 [00:07<00:38,  2.94s/trial, best loss: 5.370086069268862]/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/m

In [18]:
# Q6. Promote the best model to the model registry

# The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, 
# you'll promote the best model to the model registry. We have prepared a script called register_model.py, 
# which will check the results from the previous step and select the top 5 runs.

! python ./scripts/hw-week-2/register_model.py --data_path ../output

# What is the test RMSE of the best model?; 5.567

2024/05/27 22:32:06 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/05/27 22:32:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/nix/store/29vig1xp5a0ma